In [1]:
import asyncio
import logging
from dotenv import load_dotenv
import os
import json
import traceback
from typing import List, Dict, Any
import time
import nest_asyncio
from pydantic import BaseModel, Field, create_model
import re
from openai import OpenAI
from llm_client.hub import LLMClientHub

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

load_dotenv()

sys_prompt_template = """You are a helpful assistant that analyzes customer summaries to identify relevant topics.
PROCESS:
1. Read the provided customer summary carefully
2. Select ONLY topics with higher relevance to the summary
3. YOU MUST USE FUNCTION CALLING for each relevant topic
4. DO NOT return free text lists - ONLY use the appropriate function tools

For each relevant topic, call the corresponding function tool.

EXAMPLES:
    
Summary 1:
"Customer is upset due to a 6-day delivery delay and a scratched phone. Support was unhelpful. They're requesting a refund or replacement. Human handover needed."

Highly Relevant Topics:
-  Delivery Problems   
- Issue not solved
- No compensation given

Summary 2:
"Customer frustrated with support agent's lengthy response times during order inquiry. Responses took 10-25 minutes each, turning a simple shipping status check into a 2-hour conversation. Customer expressed increasing irritation with the delayed communication."

Highly Relevant Topics:
- Agent replied slowly



"""

user_prompt_template = """Analyze the following summary and identify the topics being discussed: {user_message}

Return ONLY the NAMES of topics with higher relevance to this summary using function calls."""

def create_topic_schema(topic_list: List[str]) -> List[Dict]:
   schema_dict = []
   for topic in topic_list:
       # Create model name: lowercase with no special chars or spaces
       model_name = re.sub(r'[^a-zA-Z0-9]', '', topic).lower()
       # Create schema
       schema = {
           'properties': {
               'description': {
                   'description': topic,
                   'title': 'Description',
                   'type': 'string'
               }
           },
           'required': ['description'],
           'title': model_name,
           'type': 'object'
       }
       schema_dict.append(schema)
   return schema_dict

# Topic list
topics = ["Issue resolution", "Damaged product", "Poor customer support", 
         "Refund delay", "Agent communication issues", "Escalation to management"]

# Create schemas
dynamic_models = create_topic_schema(topics)

# Convert dynamic models to proper OpenAI tool format
def create_tools(schemas):
   tools = []
   for schema in schemas:
       tools.append({
           "type": "function",
           "function": {
               "name": schema["title"],
               "description": f"Identify if {schema['properties']['description']['description']} is relevant",
               "parameters": schema
           }
       })
   return tools


def create_config(dynamic_models):
   
   tools = create_tools(dynamic_models)
   
   return {
       "model": {
           "provider": "openai",
           "model": "gpt-4o-mini",
           "temperature": 0,
           "max_tokens": 1000,
           "mode": "PARALLEL_TOOLS",
           "tool_choice": "required"  
       },
       "sys_tmpl": sys_prompt_template,
       "usr_tmpl": user_prompt_template,
       "response_type": "structured_response",
       "DynamicModel": dynamic_models,
       "tools": tools  
   }

async def main():
   try:
       start_time = time.time()
       
       context = {
           "user_message": "Customer requested more information. Provided overview of Al-Nokhba Home Equipment Company: specializes in manufacturing and installing windows, doors, kitchens, wardrobes, stair barriers, and sanitary materials; experienced since 2009. Asked customer to specify what information they need."
       }
       
       config = create_config(dynamic_models)
       
       logger.debug(f"Tools: {json.dumps(config['tools'][0], indent=2)}")
       
       llm_client_hub = LLMClientHub(config)
       
       response, raw_completion = await llm_client_hub.handle_response(context)
       
       print("Response:", response)
       print(f"Execution time: {time.time() - start_time:.2f} seconds")
       
       return {"response": response, "raw_completion": raw_completion}
       
   except Exception as e:
       logger.error(f"Error during execution: {e}")
       logger.error(traceback.format_exc())
       return {"error": str(e)}

if __name__ == "__main__":
   try:
       
       nest_asyncio.apply()
       
       
       loop = asyncio.get_event_loop()
       
       if loop.is_running():
           logger.info("Event loop is already running. Using create_task instead.")
           task = asyncio.create_task(main())
           
       else:
           result = loop.run_until_complete(main())
           print("Result:", result)
           
   except Exception as e:
       logger.error(f"Error during execution: {e}")
       logger.error(traceback.format_exc())

2025-05-12 21:54:18,877 - INFO - Event loop is already running. Using create_task instead.


2025-05-12 21:54:18,882 - DEBUG - Tools: {
  "type": "function",
  "function": {
    "name": "issueresolution",
    "description": "Identify if Issue resolution is relevant",
    "parameters": {
      "properties": {
        "description": {
          "description": "Issue resolution",
          "title": "Description",
          "type": "string"
        }
      },
      "required": [
        "description"
      ],
      "title": "issueresolution",
      "type": "object"
    }
  }
}
2025-05-12 21:54:19,048 - DEBUG - Patching `client.chat.completions.create` with mode=<Mode.PARALLEL_TOOLS: 'parallel_tool_call'>
2025-05-12 21:54:19,049 - INFO - ResponseHandler initialized with client: <llm_client.clients.openai_client.OpenAICompletionClient object at 0x12e54f200> and settings: {'provider': 'openai', 'model': 'gpt-4o-mini', 'temperature': 0, 'max_tokens': 1000, 'mode': 'PARALLEL_TOOLS', 'tool_choice': 'required'}
2025-05-12 21:54:19,049 - INFO - Client and handler initialized successfully.

Response: [agentcommunicationissues(description='Customer requested more information from the support agent.'), escalationtomanagement(description='Customer asked for specific information regarding products and services.')]
Execution time: 3.62 seconds
